In [1]:
# Install Required Packages & Import Dependencies
%%capture
!pip install transformers torch pandas numpy tqdm scikit-learn nltk
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from typing import Tuple

class MathProblemSolver:
    def __init__(self, model_name="t5-base", checkpoint_path=None):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        if checkpoint_path:
            self.load_checkpoint(checkpoint_path)

        print(f"Using device: {self.device}")

    def load_checkpoint(self, checkpoint_path: str):
        """Load a trained model checkpoint."""
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()  # Set the model to evaluation mode
        print("Loaded trained model checkpoint successfully!")

    def solve(self, problem: str) -> str:
        """Generate equation for a given word problem."""
        self.model.eval()
        input_text = f"Generate equation: {problem}"

        inputs = self.tokenizer(
            input_text,
            max_length=256,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        ).to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=128,
                num_beams=5,
                length_penalty=1.0,
                early_stopping=True,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

        equation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return equation

    def solve_and_compute(self, problem: str) -> Tuple[str, float]:
        """Generate equation and compute the result for a given word problem."""
        equation = self.solve(problem)
        equation_to_solve = equation.split('=')[0].strip()

        try:
            result = eval(equation_to_solve)
            return equation, result
        except:
            return equation, "Unable to compute result"


In [2]:
# Mount Google Drive and Initialize Solver
from google.colab import drive
drive.mount('/content/drive')

# Initialize the solver with your trained model
checkpoint_path = '/content/drive/MyDrive/AWPSS/AWPSS_BEST_MODEL.pt'
solver = MathProblemSolver(checkpoint_path=checkpoint_path)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

<ipython-input-1-f3d320fa741f>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.device)


Loaded trained model checkpoint successfully!
Using device: cpu


In [3]:
# Install Required Packages for Streamlit
!pip install streamlit pyngrok transformers torch
!pip install streamlit pyngrok transformers torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.7 MB/s eta 0:00:00


In [4]:
# Import Libraries
from pyngrok import ngrok
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Create a Streamlit Config File for Light Mode
!mkdir -p /root/.streamlit  # Ensure the full path
with open("/root/.streamlit/config.toml", "w") as config_file:
    config_file.write("""
    [theme]
    base="light"
    """)

# Define the Math Problem Solver Class and Write Streamlit Code to app.py
app_code = """
import streamlit as st
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Define Math Problem Solver Class
class MathProblemSolver:
    def __init__(self, model_name="t5-base", checkpoint_path=None):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

        if checkpoint_path:
            self.load_checkpoint(checkpoint_path)

    def load_checkpoint(self, checkpoint_path: str):
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()

    def solve(self, problem: str) -> str:
        self.model.eval()
        input_text = f"Generate equation: {problem}"
        inputs = self.tokenizer(input_text, max_length=256, padding='max_length', truncation=True, return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=256, num_beams=5, length_penalty=1.0, early_stopping=True, no_repeat_ngram_size=2, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def solve_and_compute(self, problem: str):
        equation = self.solve(problem)
        equation_to_solve = equation.split('=')[0].strip()
        try:
            result = eval(equation_to_solve)
            return equation, result
        except:
            return equation, "Unable to compute result"

# Streamlit App Code
def main():
    checkpoint_path = '/content/drive/MyDrive/AWPSS/AWPSS_BEST_MODEL.pt'
    solver = MathProblemSolver(checkpoint_path=checkpoint_path)

    st.set_page_config(
        page_title="Arithmetic Word Problem Solving System",
        page_icon="🧮",
        layout="centered",
        initial_sidebar_state="collapsed",
    )

    st.markdown("<h1 style='color:black;'>🧮 Arithmetic Word Problem Solving System</h1>", unsafe_allow_html=True)
    st.write("Enter a word problem, and the model will generate the equation and compute the answer.")

    # Apply custom styles to make the input box and button white with black borders
    st.write(
        \"\"\"
        <style>
        .stApp {
            background-color: #FFFFFF;
            color: #000000;
        }
        div.stButton > button {
            background-color: #FFFFFF;
            color: #000000;
            border: 1px solid #000000;
        }
        textarea {
            background-color: #FFFFFF;
            color: #000000;
            border: 1px solid #000000;
        }
        </style>
        \"\"\",
        unsafe_allow_html=True,
    )

    # Input field for user word problem
    user_input = st.text_area("Enter your word problem here:", "")

    if st.button("Solve"):
        if user_input:
            equation, result = solver.solve_and_compute(user_input)
            st.markdown(f"**Generated Equation:** <span style='color:blue;'>{equation}</span>", unsafe_allow_html=True)
            st.markdown(f"**Result:** <span style='color:green;'>{result}</span>", unsafe_allow_html=True)
        else:
            st.write("Please enter a word problem.")


if __name__ == "__main__":
    main()
"""

# Write the app code to `app.py`
with open("app.py", "w") as file:
    file.write(app_code)


In [5]:
!pgrep ngrok | xargs kill


Usage:
 kill [options] <pid> [...]

Options:
 <pid> [...]            send signal to every <pid> listed
 -<signal>, -s, --signal <signal>
                        specify the <signal> to be sent
 -q, --queue <value>    integer value to be sent with the signal
 -l, --list=[<signal>]  list all signal names, or convert one to a name
 -L, --table            list all signal names in a nice table

 -h, --help     display this help and exit
 -V, --version  output version information and exit

For more details see kill(1).


In [6]:
# Install ngrok authtoken
!ngrok authtoken 2oeyke9lb3kXVw0DQj84GtwgxEH_21nqRahuhnULEL3Lk6Zsw

# Launch ngrok with HTTP tunnel on port 8501 for Streamlit
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Streamlit app running at: {public_url}")

# Run the Streamlit app
!streamlit run app.py &>/dev/null &

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app running at: NgrokTunnel: "https://7929-35-245-131-213.ngrok-free.app" -> "http://localhost:8501"
